# FRB/US, modelconsistent expectations
This notebook is based on the the FRB/US model packages located here (https://www.federalreserve.gov/econres/us-models-package.htm). The packages provides an Eviews (https://www.eviews.com/) implementation. The Eviews modelspecification has been transformed to ModelFlow BusinessLogic specification. 

FRB/US comes in two flawors: **VAR** in which expectaions are generated by backward looking equations, and **MCE** in which expectation are based on leaded values.

This notebook implement the **MCE** flavour. The presence of leaded variable implies that the model has to be solved for all quarters in one go. This is done by a stacked newton method

The **VAR** model which has only lagged variables can be solved for each quarter in turn. 

## Import stuff

In [19]:
#This is code to manage dependencies if the notebook is executed in the google colab cloud service
if 'google.colab' in str(get_ipython()):
  import os
  os.system('apt -qqq install graphviz')
  os.system('pip -qqq install ModelFlowIb ipysheet  --no-dependencies ')

In [20]:
from modelclass import model
import modelmf

## Load the model and a baseline databank

In [21]:
mfrbusmce,basedf = model.modelload('mfrbusmce.pcim')

file read:  C:\modelflow manual\papers\modelbook\content\Federal Reserve Board - US model\mfrbusmce.pcim


## Run the baseline 

In [22]:
baseres = mfrbusmce(basedf,'2020q1','2040q4', silent=0)

Prepare model for calculate derivatives for Newton solver
Model                           : FRB/US Forward expectations
Number of endogeneus variables  : 285
Number of derivatives           : 1359
Calculating new derivatives and create new stacked Newton solver
 all lags and leads calculated  
Iteration  0 Sum of distances            0.001428857786
Iteration  1 Sum of distances            0.000000026509
Solved in 1 iterations
FRB/US Forward expectations solved  


## Make a number of experiments, and run them 

In [23]:
mfrbusmce.keep_solutions = {}
for shock in [0.0, 0.005 ,0.01 ,0.1]:
    altdf=baseres.upd(f'rffintay_aerr + {shock}')       
        
    with model.timer(f'newton all periods,shock = {shock:7} '):
        altres = mfrbusmce(altdf,keep=f'Shock={shock}',silent=1)

newton all periods,shock =     0.0  took       :         0.0781574249 Seconds
newton all periods,shock =   0.005  took       :         0.3075926304 Seconds
newton all periods,shock =    0.01  took       :         0.4265885353 Seconds
 all lags and leads calculated  
newton all periods,shock =     0.1  took       :         9.650 Seconds


## Display results 

In [24]:
mfrbusmce.vline = [('2020q1','   Projection')]
with mfrbusmce.set_smpl('2015q1','2040q4'):
    mfrbusmce.keep_viz('RFF')

Output()